## Imports and Database Connection

In [ ]:

import sqlite3
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

conn = sqlite3.connect('room_data.db')


## NAIA method for every room in room_ids

In [ ]:
room_ids = [1, 2, 3, 4, 6, 7, 8, 9, 20, 21, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 311, 51, 56, 57, 59, 60, 87, 88, 89, 90, 91, 92, 93, 94, 95, 104, 105, 115, 116, 117, 297, 261]

for room_id in room_ids:
    print(f"\n=== Processing room_id: {room_id} ===")

    query = f"""
    SELECT room_id, timestamp, temperature, airquality, daylight, light
    FROM sensor_data_history
    WHERE room_id = {room_id}
    ORDER BY timestamp ASC
    """
    df = pd.read_sql_query(query, conn, parse_dates=['timestamp'])

    if df.empty:
        print("No data available.")
        continue

    df = df.set_index('timestamp')
    hourly = df.resample('1h').mean().dropna()
    hourly['date'] = hourly.index.date
    hourly['hour'] = hourly.index.hour

    daily_patterns = hourly.pivot_table(
        values='airquality',
        index=['date', 'hour'],
        aggfunc='mean'
    ).unstack()
    daily_patterns = daily_patterns.dropna()

    if daily_patterns.empty:
        print("Not enough hourly data.")
        continue

    daily_patterns.columns = daily_patterns.columns.droplevel(0)
    daily_patterns.index = pd.to_datetime(daily_patterns.index)

    def iterative_clustering(data, initial_k=3, max_iters=5, var_threshold=0.5):
        k = initial_k
        X = data.values
        for i in range(max_iters):
            kmeans = KMeans(n_clusters=k, random_state=0).fit(X)
            labels = kmeans.labels_
            centers = kmeans.cluster_centers_
            dispersions = []
            for j in range(k):
                cluster_points = X[labels == j]
                dist = np.linalg.norm(cluster_points - centers[j], axis=1)
                dispersions.append(dist.std())
            worst = np.argmax(dispersions)
            if dispersions[worst] < var_threshold:
                break
            k += 1
        return KMeans(n_clusters=k, random_state=0).fit(data)

    cluster_model = iterative_clustering(daily_patterns, initial_k=3)
    labels = cluster_model.labels_
    daily_patterns['cluster'] = labels

    outlier_hours = []
    for cluster in np.unique(labels):
        cluster_data = daily_patterns[daily_patterns.cluster == cluster].iloc[:, :-1]
        center = cluster_model.cluster_centers_[cluster]
        dists = np.linalg.norm(cluster_data - center, axis=1)
        Q1, Q3 = np.percentile(dists, [25, 75])
        iqr = Q3 - Q1
        lower, upper = Q1 - 1.5*iqr, Q3 + 1.5*iqr
        mask = (dists < lower) | (dists > upper)
        timestamps = cluster_data.index[mask]
        outlier_hours.extend(timestamps)

    print("Outlier hours:", outlier_hours)

    pca = PCA(n_components=2)
    proj = pca.fit_transform(daily_patterns.iloc[:, :-1])
    plt.figure(figsize=(8,6))
    plt.scatter(proj[:,0], proj[:,1], c=labels, cmap='tab10', alpha=0.6)
    for d in outlier_hours:
        if d in daily_patterns.index:
            idx = daily_patterns.index.get_loc(d)
            plt.scatter(proj[idx,0], proj[idx,1], c='k', marker='x')
    plt.title(f'Room {room_id}: Clustered Hourly Patterns with Outliers')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    plt.show()

    # Use the last day in the dataset
    latest_ts = daily_patterns.index[-1]
    row = daily_patterns.loc[latest_ts]
    row_no_cluster = row.drop('cluster')
    new_day = pd.DataFrame([row_no_cluster], index=[latest_ts])

    def validate_new(patterns, cluster_model, data_with_labels):
        results = {}
        centers = cluster_model.cluster_centers_
        radii = {}
        for c in np.unique(data_with_labels.cluster):
            pts = data_with_labels[data_with_labels.cluster == c].iloc[:, :-1].values
            radii[c] = np.max(np.linalg.norm(pts - centers[c], axis=1))
        regs = {}
        for c in np.unique(data_with_labels.cluster):
            subset = data_with_labels[data_with_labels.cluster == c]
            X = np.arange(len(subset)).reshape(-1, 1)
            y = subset.iloc[:, :-1].mean(axis=1).values
            lr = LinearRegression().fit(X, y)
            regs[c] = lr

        for date, row in patterns.iterrows():
            x_mean = row.mean()
            dists = np.linalg.norm(centers - row.values, axis=1)
            c = np.argmin(dists)
            cond1 = dists[c] <= radii[c]
            y_pred = regs[c].predict(np.array([[len(data_with_labels)]]))[0]
            tol = 1.5 * np.std(data_with_labels[data_with_labels.cluster == c].iloc[:, :-1].mean(axis=1))
            cond2 = abs(x_mean - y_pred) <= tol
            results[date] = 'inlier' if cond1 and cond2 else 'outlier'
        return results

    validation_results = validate_new(new_day, cluster_model, daily_patterns)
    print("Validation result for last day (Phase 2):", validation_results)

    # Store anomalies in the database
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS anomalies (
            room_id INTEGER,
            timestamp DATETIME,
            temperature REAL,
            airquality REAL,
            daylight REAL,
            light INTEGER,
            source_model TEXT,
            PRIMARY KEY (room_id, timestamp, source_model)
        )
    """)

    for ts in outlier_hours:
        ts_data = df.loc[df.index.floor('h') == ts]
        for i, row_data in ts_data.iterrows():
            cursor.execute("""
                INSERT OR IGNORE INTO anomalies (room_id, timestamp, temperature, airquality, daylight, light, source_model)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                room_id,
                i.isoformat(),
                row_data.get('temperature'),
                row_data.get('airquality'),
                row_data.get('daylight'),
                int(row_data.get('light')) if not pd.isna(row_data.get('light')) else None,
                'NAIA'
            ))

    for val_ts, val_result in validation_results.items():
        if val_result == 'outlier':
            val_data = df.loc[df.index.floor('h') == val_ts]
            for i, row_data in val_data.iterrows():
                cursor.execute("""
                    INSERT OR IGNORE INTO anomalies (room_id, timestamp, temperature, airquality, daylight, light, source_model)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                """, (
                    room_id,
                    i.isoformat(),
                    row_data.get('temperature'),
                    row_data.get('airquality'),
                    row_data.get('daylight'),
                    int(row_data.get('light')) if not pd.isna(row_data.get('light')) else None,
                    'NAIA'
                ))

    conn.commit()


## Additional visualization of the results

In [ ]:
# Load anomalies and sensor data
anomalies_df = pd.read_sql_query("""
    SELECT * FROM anomalies
    WHERE source_model = 'NAIA'
""", conn, parse_dates=['timestamp'])

last_day = new_day.index[0].date()
last_day_data = df[df.index.date == last_day]
last_day_data.loc[:, 'hour'] = last_day_data.index.hour
last_day_data.loc[:, 'result'] = 'inlier'

if list(validation_results.values())[0] == 'outlier':
    last_day_data.loc[:, 'result'] = 'outlier'

plt.figure(figsize=(6, 4))
sns.countplot(data=last_day_data, x='result', hue='result', palette='Set2', legend=False)
plt.title(f'Validation Outcome for Last Day ({last_day})')
plt.xlabel('Result')
plt.ylabel('Number of Records')
plt.show()

room_validation_summary = pd.DataFrame({
    'room_id': [room_id],
    'date': [last_day],
    'result': list(validation_results.values())
})
print("Validation Outcome Summary")
display(room_validation_summary)


anomaly_counts = anomalies_df.groupby('room_id').size().reset_index(name='outlier_count')
plt.figure(figsize=(12, 5))
sns.barplot(data=anomaly_counts, x='room_id', y='outlier_count', palette='crest')
plt.title('Total Anomalies per Room (NAIA Model)')
plt.xlabel('Room ID')
plt.ylabel('Outlier Count')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()
